In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from pathlib import Path
import matplotlib.patches as patches
from dataset import CardiacDataset
from model import CardiacDetectionModel

In [3]:
train_dataset = CardiacDataset("rsna_heart_detection.csv", "train_subjects.npy", "Processed_Heart_Detection/train", 0.49, 0.25)
val_dataset = CardiacDataset("rsna_heart_detection.csv", "val_subjects.npy", "Processed_Heart_Detection/val", 0.49, 0.25)

batch_size = 16
num_workers = 4

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = False)
